# Modifications de résultats des sondages

## Import préliminaires des librairies

In [1]:
import re
import numpy as np
import pandas as pd
import locale 
from locale import atof
# Régression linéaire
from sklearn.linear_model import LinearRegression

# Read files
import PyPDF2
from pdfminer.high_level import extract_text

# Perso
from importlib.machinery import SourceFileLoader
tools = SourceFileLoader('methods', '../methods.py').load_module()
from methods import *

In [2]:
fp="rapport/2020-03-04.pdf"
text = extract_text(fp,page_numbers=[16])
question1="Êtes-vous  satisfait  ou  mécontent  d"
question2="Êtes-vous satisfait ou mécontent d"
# print(text.find(question1))
# print(text.find(question2))
# print(len(question2))
print(re.sub('\.\.\. (?![\n])','...\n',text))

Le classement des personnalités
Auprès des sympathisants de la gauche

Question : Pour chacune des personnalités suivantes, indiquez si vous en avez une image positive ou une image négative ?
En %

S/T
« Positive »

Evolution
sur 1 mois

22

17

46

36

37
39
39
41

38
30

26

28
30
20

Nicolas Hulot
Christiane Taubira
Olivier Besancenot
Benoît Hamon
François Hollande
Ségolène Royal
Anne Hidalgo
Jean-Luc Mélenchon
Yannick Jadot
Bernard Cazeneuve
Jean-Yves Le Drian
François Ruffin
Xavier Bertrand
Marlène Schiappa
Cédric Villani
Olivier Faure
Agnès Buzyn
François Baroin
François Bayrou
Rachida Dati
Olivier Véran
Gérard Larcher
Valérie Pécresse
Bruno Le Maire
Muriel Pénicaud
Nicolas Sarkozy
Marine Le Pen
Christophe Castaner
Jean-Michel Blanquer
Marion Maréchal
Gérald Darmanin
Christian Jacob
Nicolas Dupont-Aignan
Laurent Wauquiez

11
9
8
6
6

11
11

10

6
4

2

5
4
3

3
2
4
2
2
2
1
3
1
3
2
3
3
2
4

25
19

18
19

22
18
19
14
16
16
16
17
14

15
12

11
9
9
10
7

10

16

24

33

30

28

8

27

In [3]:
with open(fp,"rb") as pdf_file:
    pdf_reader= PyPDF2.PdfFileReader(pdf_file)
    isEncrypted = pdf_reader.getIsEncrypted()
    nbPages= pdf_reader.numPages

    # Define columns
    page_content=pdf_reader.getPage(15).extractText()
print(page_content)

10
7
6
4
4
8
7
5
6
5
3
3
2
2
2
3
1
3
2
3
3
3
3
3
2
2
2
2
3
2
2
2
1
1
36
26
22
24
24
18
19
20
18
19
20
20
21
20
20
18
20
17
18
16
16
15
15
15
16
15
15
14
12
13
12
12
13
9
20
26
20
17
20
14
16
17
19
26
24
24
24
29
28
26
32
25
23
24
24
17
21
22
22
14
23
20
19
22
21
25
21
21
15
28
19
17
18
45
39
30
33
26
21
29
22
33
34
22
23
28
24
28
42
18
22
35
17
15
19
24
25
39
19
30
24
18
19
13
33
38
34
15
19
28
24
24
32
24
31
16
16
31
24
27
33
29
15
47
39
25
43
54
41
40
41
24
46
31
41
51
Nicolas Hulot
Nicolas Sarkozy
François Baroin
Jean-Yves Le Drian
Xavier Bertrand
Marine Le Pen
Marion Maréchal
Olivier Besancenot
Christiane Taubira
Rachida Dati
Bernard Cazeneuve
Anne Hidalgo
Bruno Le Maire
François Hollande
Ségolène Royal
Valérie Pécresse
François Bayrou
Benoît Hamon
Agnès Buzyn
Marlène Schiappa
Jean-Luc Mélenchon
Yannick Jadot
Jean-Michel Blanquer
Nicolas Dupont-Aignan
Cédric Villani
Olivier Véran
Gérard Larcher
Gérald Darmanin
François Ruffin
Christophe Castaner
Christian Jacob
Laurent Wauquiez
Mur

# Extraction des données avec PyPDF2
PyPDF2 semble être constant dans la forme d'extraction des données de Elab.
La structure des données ainsi que les détails d'extraction sont précisés ci-dessous:
* Elabe fourni les données détaillées pour deux personnalités que sont le président de la République et le premier ministre. Les noms n'étant pas indiqués sur la page de données, il faut les extraire ailleurs ou les préciser à la main.
* Il y a deux pages successives de données par individus
* Les catégories principales sont indiquées en capitales
* Il existe des catégories secondaires et tertiaires sans différenciation possible. Elles seront traités comme sous catégories.
* La base de chaque catégories n'est pas précisée, il faudra donc utiliser de la régression linéaire pour tenter de les retrouver
* Les trous dans les données ne sont pas visible. Il faut un système de vérification.
* Les variations ne sont pas données

__Format des données__
* Les données sont fournis en lisant les lignes dans l'ordre
* Chaque cellule est extraite par PyPDF2 une à la fois en introduisant un saut de ligne
* Les blancs dans les données ne créént pas de fusion d'information mais disparaissent sans faire de double saut de lignes
* Le caractère '__-__' compromet les cellules et induit un ou plusieurs sauts malencontreux :
    * Si le '-' apparait au milieu d'un texte, il induit un _double saut de ligne_ (ex: 18-24 ans)
    * Si le '-' apparait à la fin d'un texte il induit un _seul_ saut de ligne supplémentaire avant sont apparition (CSP-)  
* Il faudra donc préformater le texte au préalable


## Extract content (to delete later)

In [4]:
def get_page_content(num_page,fp)->str:
    with open(fp,"rb") as pdf_file:
        pdf_reader= PyPDF2.PdfFileReader(pdf_file)
        isEncrypted = pdf_reader.getIsEncrypted()
        nbPages= pdf_reader.numPages
        return pdf_reader.getPage(num_page-1).extractText()

    # Define columns
page_content=get_page_content(2,fp)
page_content

"Interrogation\nFiche technique\n2\n\nMode de recueil et dates de terrain\nLa\nreprésentativité\nde\n\na\nété\nassurée\nselon\nla\nméthode\ndes\nquotas\nappliquée\naux\nvariables\nsuivantes\n:\nsexe,\nâge\net\nprofession\nde\n\naprès\nstratification\npar\nrégion\net\ncatégorie\n\n.\nEchantillon de\n1 007 \npersonnes représentatif de la population française âgée de 18 ans et plus.\nInterrogation par Internet les \n3 \net \n4 mars 2020\n.\n05/03/2020\nL'Observatoire politique \n\nMars 2020\n"

## Préformatage

In [23]:
##Préformatage
def preformatage(page_content)->str:
    #Remplacement des données d'age
    page_content=page_content.replace('18\n-\n24 ans','18-24 ans')
    page_content=page_content.replace('25\n-\n34 ans','25-34 ans')
    page_content=page_content.replace('35\n-\n49 ans','35-49 ans')
    page_content=page_content.replace('50\n-\n64 ans','50-64 ans')
    # Remplacement CSP-
    page_content=page_content.replace('CSP\n-','CSP-')
    # Remplacement EELV
    page_content=page_content.replace('Europe Ecologie \n-\nLes Verts','Europe Ecologie Les Verts')
    # Remplacement Front National
    page_content=page_content.replace('Front National','Le Rassemblement National')
    # Noms de candidats
    page_content=page_content.replace('Jean\n-\n','Jean-')
    # Remplacement /
    page_content=page_content.replace('/ \n','/ ')
    page_content=page_content.replace('\n/','/')
    # Remplacement Total .. en minuscule
    page_content=page_content.replace('TOTAL GAUCHE','Total Gauche')
    page_content=page_content.replace('TOTAL EN MARCHE / MODEM','Total en Marche / MODEM')
    page_content=page_content.replace('TOTAL DROITE','Total Droite')
    # Suppression fin de texte
    page_content=page_content.replace('\nLa cote de confiance du Président de la République','')
    page_content=page_content.replace('\nLa cote de confiance du Premier ministre','')
    # Suppression des notes de bas de pages
    page_content=page_content.replace(' (*)','')
    page_content=page_content.replace('(*)','')
    # Suppression du dernier saut de ligne
    page_content=page_content.rstrip('\n')
    return page_content

preformatage(page_content)


"Interrogation\nFiche technique\n2\n\nMode de recueil et dates de terrain\nLa\nreprésentativité\nde\n\na\nété\nassurée\nselon\nla\nméthode\ndes\nquotas\nappliquée\naux\nvariables\nsuivantes\n:\nsexe,\nâge\net\nprofession\nde\n\naprès\nstratification\npar\nrégion\net\ncatégorie\n\n.\nEchantillon de\n1 007 \npersonnes représentatif de la population française âgée de 18 ans et plus.\nInterrogation par Internet les \n3 \net \n4 mars 2020\n.\n05/03/2020\nL'Observatoire politique \n\nMars 2020"

## Extraction de la base

In [24]:
def extract_base(fp,num_page=2)->int:
    page_content=get_page_content(num_page,fp)
    text_deb='Echantillon de \n'
    if page_content.find(text_deb)==-1:
        text_deb='Echantillon de\n'
    text_fin='\npersonnes'
    i_deb=page_content.find(text_deb)+len(text_deb)
    i_fin=page_content.find(text_fin)
    base=int(page_content[i_deb:i_fin].rstrip().replace(' ',''))
    return base
extract_base(fp)

1007

## Création du modèle de Dataframe

In [25]:
df=pd.DataFrame(columns=['Nom','Base','Date','Categorie','Groupe','tres_positif','positif','negatif','tres_negatif','nsp','total_positif','total_negatif'])

## Extraction de page formattée

In [26]:
def extract_series(data,i,columns,nom,base,date,prv_categorie)->pd.Series:
    """Extrait une serie de data à partir de l'indice i jusqu'à la prochaine valeur non digit

    Args:
        data (list): Toutes les données dans une liste 
        i (int): Indice courant dans les données
        columns (list): Liste des noms de colonnes 
        nom (str): Nom de la personnalité
        base (int): Nombre de sondés
        date (datetime.date): Date du sondage

    Returns:
        pd.Series: La serie contenant les valeurs à stocker
        int: new index
    """
    values=[nom,base,date]
    #Catégorie
    if data[i].isupper() and '+' not in data[i] and '-' not in data[i]:
        values.append(data[i])
        i+=1
    else:
        values.append(prv_categorie)
    #Groupe
    values.append(data[i])
    # On saut la cellule qui contient le 100% et qui ne sert a rien
    i+=2
    while i<len(data) and re.match('^\d+$',data[i]):
        values.append(float(data[i]))
        i+=1
    # Vérification
    while len(values)!=len(columns):
        print(values)
        j=int(input('Où faut-il ajouter un zéro ? (1-7)'))+4
        values=values[:j]+[0.0]+values[j:]
    return pd.Series(values,index=columns),i



def extract_detailled_page(page_content,df,nom,base,date)->pd.DataFrame:
    """
    Extrait une page du pdf et l'ajoute au dataframe.
    
    Parameters
    ----------
    page_content: texte formatté de la page dot l'on souhaite extraire les données
    df: Dataframe pour stocker les données
    nom: Nom de la personnalité 
    base: Base d'étude (à compléter par regression plus tard)
    date: Date du sondage

    Returns
    -------
    Dataframe
        Dataframe df avec les nouvelles données
    """
    indice_debut=page_content.find('Ensemble')
    data=page_content[indice_debut:].split('\n')
    i=0
    cat='Ensemble'
    while i < len(data):
        if not re.match('^\d+$',data[i]):
            s,i=extract_series(data,i,df.columns,nom,base,date,cat)
            cat=s.Categorie
            df=df.append(s,ignore_index=True)
    return df


## Extraction pour une personnalité

In [27]:
def extract_detailled_perso(df,fp,page_indexes,nom,base,date)->pd.DataFrame:
    """Extrait les données sur toutes les pages pour une personnalité

    Args:
        df (pandas.Dataframe): Dataframe stockant les données
        fp (str): filepath du fichier à lire
        page_indexes (list): Liste des pages à extraire
        nom (str): Nom de la personnalité
        base (int): Nb de sondés
        date (datetime.date): Date du sondate
        inline (bool): Si True modification en ligne

    Returns:
        pd.DataFrame: Dataframe étendu des nouvelles données
    """
    for i in page_indexes:
        page_content=preformatage(get_page_content(i,fp))
        df=extract_detailled_page(page_content,df,nom,base,date)
    df=df.drop_duplicates(ignore_index=True)
    return df

        

In [28]:
df=pd.DataFrame(columns=['Nom','Base','Date','Categorie','Groupe','tres_positif','positif','negatif','tres_negatif','nsp','total_positif','total_negatif'])
date='2020-01-15'
filepath='rapport/{}.pdf'.format(date)
president='Emmanuel MACRON'
page_president=[9,10]
ministe='Edouard PHILIPPE'
page_ministre=[12,13]
base=extract_base(filepath)
df=extract_detailled_perso(df,filepath,page_president,president,base,date)
df=extract_detailled_perso(df,filepath,page_ministre,ministe,base,date)
df

['Emmanuel MACRON', 1006, '2020-01-15', 'PREFERENCE PARTISANE', 'La France Insoumise', 2.0, 4.0, 21.0, 73.0, 6.0, 94.0]
['Emmanuel MACRON', 1006, '2020-01-15', 'PREFERENCE PARTISANE', 'Parti Socialiste / Génération.s/ PRG', 12.0, 27.0, 21.0, 40.0, 39.0, 61.0]
['Emmanuel MACRON', 1006, '2020-01-15', 'PREFERENCE PARTISANE', 'Total en Marche / MODEM', 35.0, 51.0, 12.0, 2.0, 86.0, 14.0]
['Emmanuel MACRON', 1006, '2020-01-15', 'PREFERENCE PARTISANE', 'La République en Marche !', 39.0, 55.0, 5.0, 1.0, 94.0, 6.0]
['Emmanuel MACRON', 1006, '2020-01-15', 'PREFERENCE PARTISANE', 'Total Droite', 7.0, 35.0, 30.0, 28.0, 42.0, 58.0]
['Emmanuel MACRON', 1006, '2020-01-15', 'PREFERENCE PARTISANE', 'Les Républicains', 5.0, 37.0, 29.0, 29.0, 42.0, 58.0]
['Emmanuel MACRON', 1006, '2020-01-15', 'VOTE PRESIDENTIELLE 2017 (1ER TOUR)', 'Benoît HAMON', 5.0, 12.0, 37.0, 46.0, 17.0, 83.0]
['Edouard PHILIPPE', 1006, '2020-01-15', 'PREFERENCE PARTISANE', 'La France Insoumise', 3.0, 27.0, 70.0, 3.0, 97.0]
['Edouar

,Nom,Base,Date,Categorie,Groupe,tres_positif,positif,negatif,tres_negatif,nsp,total_positif,total_negatif
0,Emmanuel MACRON,1006,2020-01-15,Ensemble,Ensemble,8.0,24.0,24.0,39.0,5.0,32.0,63.0
1,Emmanuel MACRON,1006,2020-01-15,SEXE DU REPONDANT,Homme,10.0,23.0,22.0,41.0,4.0,33.0,63.0
2,Emmanuel MACRON,1006,2020-01-15,SEXE DU REPONDANT,Femme,6.0,25.0,25.0,38.0,6.0,31.0,63.0
3,Emmanuel MACRON,1006,2020-01-15,AGE DU REPONDANT,18-24 ans,7.0,23.0,24.0,36.0,10.0,30.0,60.0
4,Emmanuel MACRON,1006,2020-01-15,AGE DU REPONDANT,25-34 ans,12.0,21.0,18.0,44.0,5.0,33.0,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...
83,Edouard PHILIPPE,1006,2020-01-15,VOTE PRESIDENTIELLE 2017 (1ER TOUR),Marine LE PEN,1.0,7.0,26.0,60.0,6.0,8.0,86.0
84,Edouard PHILIPPE,1006,2020-01-15,VOTE PRESIDENTIELLE 2017 (1ER TOUR),"Abstention, vote blanc ou nul",3.0,16.0,27.0,40.0,14.0,19.0,67.0
85,Edouard PHILIPPE,1006,2020-01-15,VOTE PRESIDENTIELLE 2017 (2ND TOUR),Emmanuel MACRON,15.0,38.0,24.0,17.0,6.0,53.0,41.0
86,Edouard PHILIPPE,1006,2020-01-15,VOTE PRESIDENTIELLE 2017 (2ND TOUR),Marine LE PEN,1.0,8.0,28.0,58.0,5.0,9.0,86.0


## Correction des bases

In [29]:
for c  in df.Categorie.unique():
    print('{} :[{}]'.format(c,','.join(["'"+x+"'" for x in df[df.Categorie==c].Groupe.unique()])))

Ensemble :['Ensemble']
SEXE DU REPONDANT :['Homme','Femme']
AGE DU REPONDANT :['18-24 ans','25-34 ans','35-49 ans','50-64 ans','65 ans et plus']
PROFESSION DU REPONDANT :['Actifs','Artisan, commerçant','CSP+','Cadre, profession intellectuelle supérieure','Profession intermédiaire','CSP-','Employé','Ouvrier','Inactifs','Retraité','Autre Inactif']
CATEGORIE D'AGGLOMERATION :['Communes rurales','Communes urbaines de province','De 2.000 à moins de 20.000 hab.','De 20.000 à moins de 100.000 hab.','100.000 hab. et plus','Agglomération parisienne']
PREFERENCE PARTISANE :['Total Gauche','La France Insoumise','Europe Ecologie Les Verts','Parti Socialiste / Génération.s/ PRG','Total en Marche / MODEM','La République en Marche !','Total Droite','Les Républicains','Le Rassemblement National','Sans préférence partisane']
VOTE PRESIDENTIELLE 2017 (1ER TOUR) :['Jean-Luc MELENCHON','Benoît HAMON','Emmanuel MACRON','François FILLON','Marine LE PEN','Abstention, vote blanc ou nul']
VOTE PRESIDENTIELLE 2

In [30]:
# CategorieX, GroupeX, CategorieY, GroupeY
relations=[
    ('SEXE DU REPONDANT' ,['Homme','Femme'],'Ensemble','Ensemble'),
    ('AGE DU REPONDANT' ,['18-24 ans','25-34 ans','35-49 ans','50-64 ans','65 ans et plus'],'Ensemble','Ensemble'),
    ('PROFESSION DU REPONDANT' ,['Actifs','Inactifs'],'Ensemble','Ensemble'),
    ('PROFESSION DU REPONDANT' ,['Artisan, commerçant','CSP+','CSP-'],'PROFESSION DU REPONDANT','Actifs'),
    ('PROFESSION DU REPONDANT' ,['Cadre, profession intellectuelle supérieure','Profession intermédiaire'],'PROFESSION DU REPONDANT','CSP+'),
    ('PROFESSION DU REPONDANT' ,['Employé','Ouvrier'],'PROFESSION DU REPONDANT','CSP-'),
    ('PROFESSION DU REPONDANT' ,['Retraité','Autre Inactif'],'PROFESSION DU REPONDANT','Inactifs'),
    ("CATEGORIE D'AGGLOMERATION" ,['Communes rurales','Communes urbaines de province','Agglomération parisienne'],'Ensemble','Ensemble'),
    ("CATEGORIE D'AGGLOMERATION" ,['De 2.000 à moins de 20.000 hab.','De 20.000 à moins de 100.000 hab.','100.000 hab. et plus'],"CATEGORIE D'AGGLOMERATION",'Communes urbaines de province'),
    ('PREFERENCE PARTISANE' ,['Total Gauche','Total en Marche / MODEM','Total Droite','Le Rassemblement National','Sans préférence partisane'],'Ensemble','Ensemble'),
    ('PREFERENCE PARTISANE' ,['La France Insoumise','Europe Ecologie Les Verts','Parti Socialiste / Génération.s/ PRG'],'PREFERENCE PARTISANE','Total Gauche'),
    ('PREFERENCE PARTISANE' ,['La République en Marche !'],'PREFERENCE PARTISANE','Total en Marche / MODEM'),
    ('PREFERENCE PARTISANE' ,['Les Républicains'],'PREFERENCE PARTISANE','Total Droite'),
    ('VOTE PRESIDENTIELLE 2017 (1ER TOUR)' ,['Jean-Luc MELENCHON','Benoît HAMON','Emmanuel MACRON','François FILLON','Marine LE PEN','Abstention, vote blanc ou nul'],'Ensemble','Ensemble'),
    ('VOTE PRESIDENTIELLE 2017 (2ND TOUR)' ,['Emmanuel MACRON','Marine LE PEN','Abstention, vote blanc ou nul'],'Ensemble','Ensemble'),   
]

In [31]:
df=recompute_base(df,relations,df.columns[5:])
df[['Categorie','Base']].where(df.Nom==df.Nom.unique()[0]).groupby(by='Categorie').sum()

subXXX
SEXE DU REPONDANT
['Homme', 'Femme']
   tres_positif  positif  negatif  tres_negatif  nsp  total_positif  \
1          10.0     23.0     22.0          41.0  4.0           33.0   
2           6.0     25.0     25.0          38.0  6.0           31.0   

   total_negatif  
1           63.0  
2           63.0  
subXXX
SEXE DU REPONDANT
['Homme', 'Femme']
    tres_positif  positif  negatif  tres_negatif  nsp  total_positif  \
45           8.0     22.0     23.0          40.0  7.0           30.0   
46           8.0     21.0     29.0          33.0  9.0           29.0   

    total_negatif  
45           63.0  
46           62.0  
('SEXE DU REPONDANT', ['Homme', 'Femme'], 'Ensemble', 'Ensemble')
[402. 604.]
subXXX
AGE DU REPONDANT
['18-24 ans', '25-34 ans', '35-49 ans', '50-64 ans', '65 ans et plus']
   tres_positif  positif  negatif  tres_negatif   nsp  total_positif  \
3           7.0     23.0     24.0          36.0  10.0           30.0   
4          12.0     21.0     18.0          44.0

,Base
Categorie,
AGE DU REPONDANT,1006.0
CATEGORIE D'AGGLOMERATION,1691.0
Ensemble,1006.0
PREFERENCE PARTISANE,1520.0
PROFESSION DU REPONDANT,2555.0
SEXE DU REPONDANT,1006.0
VOTE PRESIDENTIELLE 2017 (1ER TOUR),1006.0
VOTE PRESIDENTIELLE 2017 (2ND TOUR),1006.0


In [33]:
df.head()
# pd.to_pickle(df,'data/{}.p'.format(date))

# Resultats de toutes les personnalités
## Extraction des valeurs

In [18]:

def preformatage_personnalite(page_content)->str:
    page_content=page_content.replace('\n-\n','\n-')
    page_content=page_content.replace('\n(NP*) Personnalités non testées dans la baromètre le mois précédent','')
    page_content=page_content.replace('NP*','0')
    page_content=page_content.replace('NP','0')
    return page_content

def first_index_not_numerical(list_content)->int:
    """Retourne l'index de la première valeur qui ne soit pas une chaine de caractère représentant un nombre

    Args:
        list_content (List[str]): Liste du contenant une ligne du texte par cellule

    Returns:
        int: index de la position recherchée. Retourne -1 si non trouvée
    """
    for i in range(len(list_content)):
        if not re.match('^\d+$',list_content[i]):
            return i
    return -1
    

def extract_detailled_page_all_personnalite(page_content,columns,base,date)->pd.DataFrame:
    # Données fournies dès le début de la page
    data=page_content.split('\n')
    index_names=first_index_not_numerical(data)
    # Création des données pour faire le dataframe
    values=np.array([int(x) for x in data[:index_names]]).reshape(5,-1).T
    nb_perso=values.shape[0]

    total_pos=values[:,0:2].sum(axis=1).reshape(-1,1)
    total_neg=values[:,2:4].sum(axis=1).reshape(-1,1)

    noms=data[index_names:index_names+nb_perso]
    noms=['{} {}'.format(x.split(' ')[0],x.split(' ')[1].upper()) for x in noms]
    noms=np.array(noms).reshape(-1,1)
    

    variations=np.array(data[-1-nb_perso:-1])
    variations[variations=='=']=0
    variations=variations.reshape(-1,1)

    base_np=np.array([[base]]*nb_perso)

    date_np=np.array([[date]]*nb_perso)

    cat=np.array([['Ensemble']]*nb_perso)

    groupe=np.array([['Ensemble']]*nb_perso)

    # Création du tableau numpy
    tab=np.append(noms,base_np,axis=1)
    tab=np.append(tab,date_np,axis=1)
    tab=np.append(tab,cat,axis=1)
    tab=np.append(tab,groupe,axis=1)
    tab=np.append(tab,values,axis=1)
    tab=np.append(tab,total_pos,axis=1)
    tab=np.append(tab,total_neg,axis=1)
    tab=np.append(tab,variations,axis=1)
    df=pd.DataFrame(tab,columns=columns)
    return df

def extract_personnalite(fp,num_page,columns):
    page_content=preformatage_personnalite(get_page_content(num_page,fp))
    df=extract_detailled_page_all_personnalite(page_content,columns,base,date)
    return df



In [35]:
COLUMNS_PERSO=['Nom','Base','Date','Categorie','Groupe','tres_positif','positif','negatif','tres_negatif','nsp','total_positif','total_negatif','evolution_positif']
DTYPES_PERSO={'Nom':str,'Base':float,'Date':'datetime64[ns]','Categorie':str,'Groupe':str,'tres_positif':float,'positif':float,'negatif':float,'tres_negatif':float,'nsp':float,'total_positif':float,'total_negatif':float,'evolution_positif':float}


filepath='rapport/{}.pdf'.format(date)
base=extract_base(filepath)
df_perso=extract_personnalite(filepath,16,COLUMNS_PERSO)
df_perso=df_perso.astype(DTYPES_PERSO)
df_perso.head()

# pd.to_pickle(df_perso,'data_perso/{}.p'.format(date))

,Nom,Base,Date,Categorie,Groupe,tres_positif,positif,negatif,tres_negatif,nsp,total_positif,total_negatif,evolution_positif
0,Nicolas HULOT,1006.0,2020-01-15,Ensemble,Ensemble,12.0,35.0,18.0,16.0,19.0,47.0,34.0,-5.0
1,Nicolas SARKOZY,1006.0,2020-01-15,Ensemble,Ensemble,8.0,24.0,27.0,28.0,13.0,32.0,55.0,-1.0
2,Jean-Yves LE,1006.0,2020-01-15,Ensemble,Ensemble,7.0,23.0,17.0,18.0,35.0,30.0,35.0,0.0
3,Xavier BERTRAND,1006.0,2020-01-15,Ensemble,Ensemble,6.0,24.0,21.0,18.0,31.0,30.0,39.0,1.0
4,François BAROIN,1006.0,2020-01-15,Ensemble,Ensemble,5.0,23.0,20.0,17.0,35.0,28.0,37.0,-1.0


In [156]:
df_perso.head()

,Nom,Base,Date,Categorie,Groupe,tres_positif,positif,negatif,tres_negatif,nsp,total_positif,total_negatif,evolution_positif
0,Nicolas HULOT,1003.0,2020-03-31,Ensemble,Ensemble,11.0,39.0,20.0,12.0,18.0,50.0,32.0,4.0
1,Nicolas SARKOZY,1003.0,2020-03-31,Ensemble,Ensemble,6.0,30.0,23.0,29.0,12.0,36.0,52.0,3.0
2,Olivier VÉRAN,1003.0,2020-03-31,Ensemble,Ensemble,9.0,26.0,17.0,13.0,35.0,35.0,30.0,18.0
3,Xavier BERTRAND,1003.0,2020-03-31,Ensemble,Ensemble,5.0,27.0,19.0,18.0,31.0,32.0,37.0,4.0
4,Jean-Yves LE,1003.0,2020-03-31,Ensemble,Ensemble,5.0,26.0,18.0,15.0,36.0,31.0,33.0,3.0


In [66]:

# with open("rapport/2020-10-14.pdf","rb") as pdf_file:
#     pdf_report= PyPDF2.PdfFileReader(pdf_file)
#     isEncrypted = pdf_report.getIsEncrypted()
#     nbPages= pdf_report.numPages

#     page1=pdf_report.getPage(15)
#     text=page1.extractText().replace(',','.')
#     text = extract_text(pdf_file,page_numbers=[14])

# print(text)

print(df.isnull().values.any())
df.head(10)
df.describe()
df[df.Base==0]

False


,Nom,Base,Date,Categorie,Groupe,tres_positif,positif,negatif,tres_negatif,nsp,total_positif,total_negatif


## Find mistakes and try to correct them

In [18]:
def find_fav_mistake(df):
    tfav=df['Très favorable']
    pfav=df['Plutôt favorable']
    stfav=df['ST Favorable']
    fav_df= df[np.abs(tfav+pfav-stfav)>0.11]
    return fav_df

def find_def_mistake(df):
    tdef=df['Très défavorable']
    pdef=df['Plutôt défavorable']
    stdef=df['ST Défavorable']
    def_df= df[np.abs(tdef+pdef-stdef)>0.11]
    return def_df

def find_nsp_mistake(df):
    stfav=df['ST Favorable']
    stdef=df['ST Défavorable']
    nsp=df['Nsp']
    tot_df= df[np.abs(stfav+stdef+nsp-100)>0.11]
    return tot_df

def find_mistake(df):
    fav_df= find_fav_mistake(df)
    def_df= find_def_mistake(df)
    tot_df=find_nsp_mistake(df)
    error_df=pd.concat([fav_df,def_df,tot_df])
    return error_df

In [19]:
def realign_fav(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["Très favorable","Plutôt favorable","ST Favorable"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_fav(df,i+1)
    return df

def realign_def(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["Très défavorable","Plutôt défavorable","ST Défavorable"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_def(df,i+1)
    return df

def realign_nsp(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["ST Favorable","ST Défavorable","Nsp"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_nsp(df,i+1)
    return df

    



In [20]:
def correct_mistakes(df):
    df_err=find_fav_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_fav(df_err[cond])
    df.loc[df_err.index]=df_err

    df_err=find_def_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_def(df_err[cond])
    df.loc[df_err.index]=df_err

    df_err=find_nsp_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_nsp(df_err[cond])
    df.loc[df_err.index]=df_err
    return df

In [21]:
df_err=find_mistake(df)
if len(df_err)>0:
    index=df_err.iloc[0:1].index[0]
    val=df_err.iloc[0].values[3:]
    print(val)
    correct=[]
    cols=["Base","Très favorable","Plutôt favorable","ST Favorable","Plutôt défavorable","Très défavorable","ST Défavorable","Nsp","Evolution ST Favorable","Evolution ST Défavorable"]
    while len(correct)!=10:
        correct=input()
        correct=[float(x) for x in correct.split(' ')]
    print(correct)
    df.loc[index,cols]=correct

KeyError: 'Très favorable'

In [32]:
df_err=find_mistake(df)
df_err

KeyError: 'Très favorable'

In [ ]:

# df.to_pickle('data/'+filename+'.p')